# Packages


In [22]:
import spacy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
from tqdm import tqdm
import json
import duden
import time
from collections import Counter
import pandas as pd
import csv


# Get Lyrics


- API Schlüssel und Zugangsdaten einsetzen
- Playlist IDs von Spotify suchen 
- Jeder Track der Playlist wird auf LyricsGenius gesucht
- Sollte zu einem Song kein Lyrics gefunden werden, wird die Anzahl hochgezählt
- Es wird eine JSON Datei gebildet

In [6]:
SPOTIPY_CLIENT_ID = '46da6046625f409cb53ab06bf807861d'
SPOTIPY_CLIENT_SECRET = '00efb0ab68db4f1eaf27ec53a4c1e068'
GENIUS_ACCESS_TOKEN = 'OsBvEsJ_OLKYR42CJoaA_N97GNOS44XXFk4KPPcLtCYnWmDwnnzY2RpcG_SdG25a'

In [ ]:
playlist_id = '2siI2ILJEPr88yibWLgQQl'  #Playlist: "Deutsche Songs die jeder kennt ;)"
playlist_id = '37i9dQZF1DWTWEW1zqSeEj' #Playlist: Zurück in die 90er
playlist_id ='56r5qRUv3jSxADdmBkhcz7' # Playlist: Top Hits of 2022

Ausführen von getlyrics.py

## Analysis

In [43]:
with open("./all_songs_neu.json") as f:
    songs_dict = json.load(f)

1. Spacy

Ausführen von cyspa.py

Durchlaufzeit war hier bei 3 Playlisten 1,5h

In [44]:
#NEU #Spacy muss gedownloaded werden mit : pip install -U pip setuptools wheel   und    pip install spacy 
#de_core_news_sm-3.5.0-py3-none-any.whl (muss gedownloaded werden -> https://spacy.io/models/de/)
#de_core_news_lg-3.5.0-py3-none-any.whl
#de_dep_news_trf-3.5.0-py3-none-any.whl

nlp2 = spacy.load ("de_core_news_lg")

list_of_nouns, list_of_pronouns = [],[]
playlist="Spotify"

for song_id in tqdm(range(len(songs_dict[playlist]))):

    lyrics_from_song = songs_dict[playlist][str(song_id)]["lyrics"] #hier wird aus dem der Datei mit allen Songtexten nur das Lyrics abgefragt
    doc3 = nlp2 (lyrics_from_song)

        # Ausgabe lediglich von Wörtern, die Nomen und Pronomen sind:
    for token in doc3: 
        if token.pos_ == "NOUN":
            list_of_nouns.append(token.text)
        elif token.pos_ == "PRON":
            list_of_pronouns.append(token.text)
        else:
            pass


100%|██████████| 88/88 [00:10<00:00,  8.33it/s]


In list_of_prnouns sind alle Pronomen der Playlist abgespeichert
In list_of_nouns sind alle Nomen der Playlist abgespeichert.

TODO: hier muss aber jedes Wort noch mit dem Jahr verknüpft werden oder mit dem Song!

In [53]:
len(list_of_nouns)

6518

In [51]:
## die Pronomen und Nomen in einzelnen dict ablegen 
lon_dict = {name: list_ for name, list_ in zip(songs_dict[playlist].keys(), list_of_nouns)}
lop_dict = {name: list_ for name, list_ in zip(songs_dict[playlist].keys(), list_of_pronouns)}

In [52]:
lon_dict

{'0': 'Freunde',
 '1': 'Jahren',
 '2': 'Kneipe',
 '3': 'Ecke',
 '4': 'Bar',
 '5': 'Neunzigern',
 '6': 'Abend',
 '7': 'Liebe',
 '8': 'Freundschaft',
 '9': 'Refrain',
 '10': 'Gläser',
 '11': 'Stadt',
 '12': 'Leben',
 '13': 'Wand',
 '14': 'Momente',
 '15': 'Dorf',
 '16': 'Tresen',
 '17': 'Spaß',
 '18': 'Leben',
 '19': 'Refrain',
 '20': 'Kneipe',
 '21': 'Ecke',
 '22': 'Licht',
 '23': 'Schnaps',
 '24': 'Kippen',
 '25': 'Kneipe',
 '26': 'Ecke',
 '27': 'Licht',
 '28': 'Licht',
 '29': 'Refrain',
 '30': 'Uhr',
 '31': 'LebenEmbed',
 '32': 'ContributorsEinmal',
 '33': 'Welt',
 '34': 'Songtext',
 '35': 'Welt',
 '36': 'Intro',
 '37': 'One',
 '38': 'One',
 '39': 'Baby',
 '40': 'Sorgen',
 '41': 'Geld',
 '42': 'Hand',
 '43': 'Welt',
 '44': 'Welt',
 '45': 'Part',
 '46': 'Ost',
 '47': 'Jackpot',
 '48': 'Bord',
 '49': 'Tag',
 '50': 'Laufe',
 '51': 'Park',
 '52': 'Geld',
 '53': 'Brot',
 '54': 'Kaviar',
 '55': 'Champagner',
 '56': 'Schampus',
 '57': 'Baby',
 '58': 'erfüll',
 '59': 'Wunsch',
 '60': 'Handkus

2. Daten sortieren



NOMEN

Json-Datei wird erstellt, um eine Liste mit allen Nomen und zugehörigen Artikeln zu bekommen. (Durchlaufzeit 28 min)

In [24]:
scrape_data = False # decide, whether to scrape full data again or read an exisiting json file.

Einzelne Json-Dateien für Nomen (m,w,n) erstellen.

# Title tbd

- TODO Sonderzeichen in strings?

### Read files

In [3]:
### Im folgenden erfolgt die Abfrage mit der genderneutralen Wortliste und der Nomenliste


gender_neu = pd.read_csv("./gender_neu_Nomen.csv",sep=";",header=None,encoding="utf8")
gender_neu = gender_neu.iloc[:, 0]
gender_neu.columns = ["Wort"]
gender_neu = list(gender_neu)

with open("./Nomen_sortiert.json", "r") as f:
    nomen_sortiert = json.load(f)

In [10]:
complete_df = pd.DataFrame(columns=["Geschlecht","Wort","neutral"])
tmp = []
playlist='Prototyp'
for id in nomen_sortiert[playlist].keys():
    for g in nomen_sortiert[playlist][id].keys(): # geschlecht
        w = nomen_sortiert[playlist][id][g] # wort
        is_gender_neutral = True if w in gender_neu else False

        complete_df.loc[len(complete_df)] = [g, w, is_gender_neutral]
        tmp.append(g)

In [11]:
freq = Counter(tmp)
freq

Counter({'Maskuline Nomen': 1195,
         'Feminine Nomen': 1092,
         'Neutrale Nomen': 711})

In [14]:
complete_df.head()

,Geschlecht,Wort,neutral
0,Feminine Nomen,Kneipe,False
1,Feminine Nomen,Ecke,False
2,Maskuline Nomen,Abend,False
3,Feminine Nomen,Liebe,False
4,Feminine Nomen,Freundschaft,False


In [15]:
complete_df.to_json("./test.json")

In [ ]:
complete_df.loc[complete_df["neutral"] == True, :]
complete_df

In [94]:
with open('Example.csv', 'w', newline = '') as csvfile:
    my_writer = csv.writer(csvfile, delimiter = ' ')
    my_writer.writerow(complete_df)

# Pronomen

In [ ]:
### Wird bestimmt auch nicht mehr benötigt!!! 


result_pronomen ={}

id=0
for playlist in lop_dict.keys():
    pronomen_playlist = {}
    for word in tqdm(lop_dict[playlist]):
        #männlich
        if word.lower() in ["er", "ihn", "derjenige", "derselbe","wer", "wen", "wem"]:
            pronomen_playlist[str(id)]={"Maskuline Pronomen": word}
            id +=1
        
        #weiblich
        if word.lower() in ["sie", "ihre", "dieselbe"]:
            pronomen_playlist[str(id)]={"Feminine Pronomen": word}
            id +=1
      
    result_pronomen[playlist]=pronomen_playlist
with open('Pronomen_sortiert.json', 'w') as fp:
    json.dump(result_pronomen, fp, indent=2)


In [ ]:
list_ = []
for id in result_pronomen["Songs22"].keys():
    list_.append(str(result_pronomen["Songs22"][id].keys()))
list_

In [ ]:
freq = Counter(list_)
freq

In [191]:
df = pd.read_json('Pronomen_sortiert.json')
df.to_csv('PRONOMEN.csv')

In [15]:
d=pd.read_csv('PRONOMEN.csv')
d=d.pop('Prototyp')
d.to_csv('PRONOMEN_neu.csv')

In [16]:

PD= pd.json_normalize(df['Prototyp'])
PD1= pd.json_normalize(df['Songs90er'])
PD2= pd.json_normalize(df['Songs22'])


In [ ]:
print()

In [ ]:
##JSON IN CSV umgewandelt 

df1 = pd.read_json('Liste_Pronomen.json')
df1.to_csv('PRONOMEN_L.csv')

df2 = pd.read_json('Nomen_sortiert.json')
df2.to_csv('Nomen_L.csv')